Implementing Q table training for 3 qubits with actions extended to cnot gates

In [1]:
from numpy import *
import cmath
from random import randrange
import pandas as pd
import time
from IPython.display import clear_output

In [2]:
a = 1
b = 1
one = array([[int(0)], [int(1)]])
zero = array([[int(1)], [int(0)]])
psi = a*zero + b*one

psi3 = kron((kron(psi,zero)), zero)
psi3

array([[1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0]])

In [6]:
right = (CNOT13.dot(CNOT12)).dot(psi3)

In [376]:
CNOT13 = array([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,0,0,1],[0,0,0,0,0,0,1,0]])
CNOT12 = array([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0]])
Toff = array([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1],[0,0,0,0,0,0,1,0]])
X = array([[0 , 1], [1, 0]])
Z = array([[1,0],[0,-1]])
Iden = array([[1 , 0], [0, 1]])
Iden3 = array([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1]])
X1 = kron(kron(X, Iden), Iden)
X2 = kron(kron(Iden, X), Iden)
X3 = kron(kron(Iden, Iden), X)
Z1 = kron(kron(Z, Iden), Iden)
Z2 = kron(kron(Iden, Z), Iden)
Z3 = kron(kron(Iden, Iden), Z)

In [418]:
list2 = [Iden3,CNOT13,CNOT12,X1,X2,X3,Z1,Z2,Z3]

In [378]:
len(list2)

10

In [5]:
def probability(guess, right):
    summ = 0
    for i in range(0,8):
        diff = (guess[i] - right[i])**2
        summ+= diff
    score = 1 - (summ)/8
    return(score)  

Apply gates with certain probabilities - adjust probabilities? Begin with a default value and update based on Q value
Use probabilities to select the next action given the input state (optimal action)

Cost for each new action also - want to reach goal as quickly as possible(as little moves as possible)

Rewards are known for being in certain states (e.g if we obtain the expected result its a reward of +1) the value of being in other states is figured out through policy and exploration

Each state-action pair has its own value (Q-value) and the poilcy selects the max value - thus deciding the correct(optimal) action for each state

Insert different input states with corresponding expected outcomes

Dataframe try

- First set up the dataframe with the correct columns and some possible states

In [419]:
Q_table = pd.DataFrame()

In [420]:
for i in range(0,len(list2)):
    new4 = array(list2[i]).dot(psi3)
    for m in range(0,len(list2)):
        gate = m
        new = (list2[m].dot(new4))
        Q = pd.DataFrame({'state':[new4], 'string' : str(new4), 'gate':gate, 'new': str(new), 'Q':0}, index=[0])
        Q_table = Q_table.append(Q)
#Q_table = Q_table.reset_index(drop=True)

It's easier to call upon cells in string format rather than array format

In [421]:
Q_table = Q_table.astype({'state': str})

In [422]:
Q_tablestr = Q_table.drop(['state'], axis = 1)
Q_tablestr = Q_tablestr.drop_duplicates()
Q_tablestr = Q_tablestr.reset_index(drop=True)

Set variables - these may need to be played about with more to find optimal values

In [423]:
table = Q_tablestr.copy()
#table

In [450]:
alpha = 0.5 #how much we update Q for every new action
gamma = 0.99 #discount factor 

num_repeats = 300
#table.loc[table.string == str(right), 'Q'] = 1

In [454]:
for repeats in range(0,num_repeats):
    psiold = psi3
    reward = 0
    for t in range(0,5): #set the max number of steps taken
        reward += 0.5 #add more penalty with each step
        
        sec = table[table['string']==str(psiold)] #choose the part of the dataframe that contains the current state
        
        gather = pd.DataFrame()
        for n in range(0,(len(sec))): #iterate through the rows that contain the current state
            for z in range(0,len(list2)): # find the action associated with the state in each row
                if(sec.iloc[n]['gate']== z):
                    apply = array(list2[z])
                    zed = z
                    
            psinew = apply.dot(psiold) #create new state by applying action
                
                
            news = table[table['string']==str(psinew)] #choose part of dataframe that contains the new state
            
            if len(news) == 0: #if the new state is not in the dataframe, insert it with all the possible actions
                for s in range(0,len(list2)):
                    lad = list2[s].dot(psinew)
                    news = pd.DataFrame({'string' : str(psinew), 'gate':s, 'new':str(lad), 'Q':0}, index=[0])
                    table = table.append(news)
                    
            maxQ = max(news['Q']) #find the max Q value the new state
            gather = gather.append(news).drop_duplicates() #create a dataframe of all possible new states
            plus = 0
            if (probability(psiold, right)) == 1: #insert a reward if the new state is the one we want
                plus = 1 
                    
            table.loc[(table.string == str(psiold)) & (table.gate == zed), 'Q'] += alpha*((plus - reward) + gamma*(maxQ) - sec.iloc[n]['Q'])
            #update the Q value of the old state based on the rewards given by the new state
        if plus == 1: #If we have reached the state we want we can stop
            print('goal reached')
            break
        else:
            top = gather.loc[(gather.Q == max(gather['Q']))] #collect the max Q value from all possible new states
            toparray =[] #sort these into an array with their associated action
            actarray =[]
            for o in range(0,len(top)):
                for p in range(0,len(list2)):
                    if (str((array(list2[p])).dot(psiold)) == (top.iloc[o]['string'])):
                        toparray.append((array(list2[p])).dot(psiold))
                        actarray.append(p)
        
            garray =[] #create an array of all possible new states with associated actions
            actg = []
            for a in range(0,len(top)):
                for b in range(0,len(list2)):
                    if (str((array(list2[b])).dot(psiold)) == (gather.iloc[a]['string'])):
                        garray.append((array(list2[b])).dot(psiold))
                        actg.append(b)
        
            q = randrange(100)
            if q > 70: #Create a percentage chance that the next state will be one with a max Q value (expoitation) or a state obtained from a random action (exploration)
                r = randrange(len(toparray))
                psiold = toparray[r]
                action = actarray[r]
            else:
                r = randrange(len(garray))
                psiold = garray[r]
                action = actg[r]
        
        #print(str(t) + 'th state is ' + str(psiold) + ' after action ' + str(action))
    print(repeats)

goal reached
0
1
goal reached
2
3
goal reached
4
goal reached
5
6
7
goal reached
8
9
10
11
12
13
14
15
16
goal reached
17
goal reached
18
19
goal reached
20
goal reached
21
goal reached
22
23
goal reached
24
goal reached
25
26
27
28
29
30
31
32
33
34
35
36
37
38
goal reached
39
goal reached
40
41
42
43
goal reached
44
45
goal reached
46
goal reached
47
goal reached
48
49
50
goal reached
51
goal reached
52
53
54
55
goal reached
56
57
goal reached
58
goal reached
59
60
61
goal reached
62
goal reached
63
goal reached
64
65
66
goal reached
67
68
69
70
71
72
73
74
75
76
77
78
goal reached
79
80
81
goal reached
82
goal reached
83
84
85
86
87
88
89
goal reached
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
goal reached
106
107
goal reached
108
109
110
goal reached
111
goal reached
112
goal reached
113
114
goal reached
115
116
goal reached
117
118
119
120
121
122
goal reached
123
goal reached
124
125
126
127
goal reached
128
goal reached
129
goal reached
130
goal reached
131
132
goal r

In [342]:
table

,string,gate,Q
0,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],0,0.187000
1,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],1,0.187000
2,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],2,-0.273712
3,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],3,-0.234200
4,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],4,0.187000
...,...,...,...
0,[[0]\n [1]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],2,0.000000
0,[[0]\n [1]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],3,0.000000
0,[[0]\n [1]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],4,0.000000
0,[[0]\n [1]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],5,0.000000


In [445]:
table.sort_values(by='Q', ascending =False).head(50)

,string,gate,new,Q
0,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]],0,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]],25.938698
0,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]],2,[[1]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]],25.314178
11,[[1]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]],2,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]],25.229167
1,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],1,[[1]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]],24.560881
9,[[1]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]],0,[[1]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]],24.558481
16,[[1]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]],7,[[1]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]],24.546351
0,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],0,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],24.016055
10,[[1]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]],1,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],24.000801
7,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],7,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],23.957691
3,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],3,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],23.957691


In [453]:
#table[table['string'] == str(psi3)]
table[table['string'] == '[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]]']

,string,gate,new,Q
18,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]],0,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]],-0.473750
19,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]],1,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]],-0.350000
20,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]],2,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],-0.547741
21,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]],3,[[0]\n [0]\n [1]\n [0]\n [1]\n [0]\n [0]\n [0]],-0.449000
22,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]],4,[[0]\n [0]\n [1]\n [0]\n [1]\n [0]\n [0]\n [0]],-0.449000
23,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]],5,[[0]\n [1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]],-0.350000
24,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]],6,[[ 1]\n [ 0]\n [ 0]\n [ 0]\n [ 0]\n [ 0]\n [-1...,-0.449000
25,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]],7,[[ 1]\n [ 0]\n [ 0]\n [ 0]\n [ 0]\n [ 0]\n [-1...,-0.449000
26,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]],8,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]],-0.498252


In [449]:
oppa = table.loc[(table.string == str(psi3))]
#top = oppa.loc[(oppa.Q == max(oppa['Q']))]
lol = oppa.sort_values("Q", ascending=False)
top = lol.head(1)
for n in [0, 1, 2]:
        
    oppa1 = table.loc[(table.string == top['new'].values[n])]
    lol = oppa1.sort_values("Q", ascending=False)
    top1 = lol.head(1)
    #top1 = oppa1.loc[(oppa1.Q == max(oppa1['Q']))]
    top = top.append(top1)
    #print(top1)
        
top.head(20)

,string,gate,new,Q
1,[[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]],1,[[1]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]],24.560881
11,[[1]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]],2,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]],25.229167
0,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]],0,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]],25.938698
0,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]],0,[[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]],25.938698


Run a few times to get different Q values and compare the Q values for each state action pair

In [458]:
alpha = 0.5 #how much we update Q for every new action
gamma = 0.99 #discount factor 

num_iter = 3
num_repeats = 100
penalty = 0.5

In [459]:
for iter in range(0,num_iter):
    table = Q_tablestr.copy()
    print(str(iter) + ' has begun')
    for repeats in range(0,num_repeats):
        psiold = psi3
        reward = 0
        for t in range(0,5): #set the max number of steps taken
            reward += penalty #add more penalty with each step
        
            sec = table[table['string']==str(psiold)] #choose the part of the dataframe that contains the current state
        
            gather = pd.DataFrame()
            for n in range(0,(len(sec))): #iterate through the rows that contain the current state
                for z in range(0,len(list2)): # find the action associated with the state in each row
                    if(sec.iloc[n]['gate']== z):
                        apply = array(list2[z])
                        zed = z
                    
                psinew = apply.dot(psiold) #create new state by applying action
                
                
                news = table[table['string']==str(psinew)] #choose part of dataframe that contains the new state
            
                if len(news) == 0: #if the new state is not in the dataframe, insert it with all the possible actions
                    for s in range(0,len(list2)):
                        lad = list2[s].dot(psinew)
                        news = pd.DataFrame({'string' : str(psinew), 'gate':s, 'new':str(lad), 'Q':0}, index=[0])
                        table = table.append(news)
                    
                maxQ = max(news['Q']) #find the max Q value the new state
                gather = gather.append(news).drop_duplicates() #create a dataframe of all possible new states
                plus = 0
                if (probability(psiold, right)) == 1: #insert a reward if the new state is the one we want
                    plus = 1 
                    
                table.loc[(table.string == str(psiold)) & (table.gate == zed), 'Q'] += alpha*((plus - reward) + gamma*(maxQ) - sec.iloc[n]['Q'])
                #update the Q value of the old state based on the rewards given by the new state
            if plus == 1: #If we have reached the state we want we can stop
                #print('goal reached')
                break
            else:
                top = gather.loc[(gather.Q == max(gather['Q']))] #collect the max Q value from all possible new states
                toparray =[] #sort these into an array with their associated action
                actarray =[]
                for o in range(0,len(top)):
                    for p in range(0,len(list2)):
                        if (str((array(list2[p])).dot(psiold)) == (top.iloc[o]['string'])):
                            toparray.append((array(list2[p])).dot(psiold))
                            actarray.append(p)
        
                garray =[] #create an array of all possible new states with associated actions
                actg = []
                for a in range(0,len(top)):
                    for b in range(0,len(list2)):
                        if (str((array(list2[b])).dot(psiold)) == (gather.iloc[a]['string'])):
                            garray.append((array(list2[b])).dot(psiold))
                            actg.append(b)
        
                q = randrange(100)
                if q > 70: #Create a percentage chance that the next state will be one with a max Q value (expoitation) or a state obtained from a random action (exploration)
                    r = randrange(len(toparray))
                    psiold = toparray[r]
                    action = actarray[r]
                else:
                    r = randrange(len(garray))
                    psiold = garray[r]
                    action = actg[r]
        
            #print(str(t) + 'th state is ' + str(psiold) + ' after action ' + str(action))
        #print(repeats)

    oppa = table.loc[(table.string == str(psi3))]
        #top = oppa.loc[(oppa.Q == max(oppa['Q']))]
    lol = oppa.sort_values("Q", ascending=False)
    top = lol.head(1)
    for n in [0, 1, 2]:
        
        oppa1 = table.loc[(table.string == top['new'].values[n])]
        lol = oppa1.sort_values("Q", ascending=False)
        top1 = lol.head(1)
        #top1 = oppa1.loc[(oppa1.Q == max(oppa1['Q']))]
        top = top.append(top1)
        #print(top1)
        
    print(top)   

0 has begun
                                             string  gate  \
2   [[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]]     2   
19  [[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]]     1   
0   [[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]]     3   
0   [[0]\n [0]\n [0]\n [1]\n [1]\n [0]\n [0]\n [0]]     0   

                                                new         Q  
2   [[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]] -4.408427  
19  [[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]] -3.084789  
0   [[0]\n [0]\n [0]\n [1]\n [1]\n [0]\n [0]\n [0]] -0.901676  
0   [[0]\n [0]\n [0]\n [1]\n [1]\n [0]\n [0]\n [0]]  0.000000  
1 has begun
                                             string  gate  \
1   [[1]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]\n [0]]     1   
11  [[1]\n [0]\n [0]\n [0]\n [0]\n [1]\n [0]\n [0]]     2   
0   [[1]\n [0]\n [0]\n [0]\n [0]\n [0]\n [0]\n [1]]     3   
0   [[0]\n [0]\n [0]\n [1]\n [1]\n [0]\n [0]\n [0]]     0   

                                            